## Import libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA, TruncatedSVD

from sklearn.preprocessing import LabelBinarizer
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.preprocessing import MaxAbsScaler, MinMaxScaler, StandardScaler

from scipy.sparse import issparse
import time

## Load data

In [2]:
data = pd.read_csv("/kaggle/input/httpparamsdataset/HttpParamsDataset-master/payload_full.csv")
data.head()

,payload,length,attack_type,label
0,c/ caridad s/n,14,norm,norm
1,"campello, el",12,norm,norm
2,40184,5,norm,norm
3,1442431887503330,16,norm,norm
4,nue37,5,norm,norm


In [3]:
train_data = data["payload"].values
label_data = data["label"].values

## Split data

In [4]:
train_data, train_cross, label_train, label_cross = train_test_split(train_data,label_data, 
                                                                     test_size=0.2, random_state = 42)

In [5]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix

n_folds = 10

kf = StratifiedKFold(n_splits=n_folds)

train_data_k_fold = []
for train_index, test_index in kf.split(train_cross, label_cross):
    X_train, X_test = train_cross[train_index], train_cross[test_index]
    y_train, y_test = label_cross[train_index], label_cross[test_index]
    train_data_k_fold.append((X_train, y_train, X_test, y_test))

# 2-gram

## RF + IG

In [6]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif

In [7]:
start_time = time.time()
vectorizer = TfidfVectorizer(min_df=0.0, analyzer="char", sublinear_tf=True, ngram_range=(2, 2), max_features=8000)
tfidf_matrix = vectorizer.fit_transform(train_data)
# pca
pca = SelectKBest(mutual_info_classif, k=256)
reduced_data = pca.fit_transform(tfidf_matrix.toarray(), label_train)

# random forest
rf_classifier = RandomForestClassifier(n_estimators=50)
rf_classifier.fit(reduced_data, label_train)
print("Thời gian chạy: ", time.time()-start_time)

Thời gian chạy:  329.2437379360199


In [8]:
total_TP = 0
total_FP = 0
total_FN = 0
total_TN = 0
import time

time_exe = 0
for X_train, y_train, X_test, y_test in train_data_k_fold:
    # tf-idf
    tfidf_matrix = vectorizer.fit_transform(X_train)
    
    # pca
    reduced_data = pca.fit_transform(tfidf_matrix.toarray(), y_train)
    
    # random forest
    rf_classifier.fit(reduced_data, y_train)
    
    start_time = time.time()
    tfidf_test = vectorizer.transform(X_test)
    reduced_test  = pca.transform(tfidf_test.toarray())
    y_pred = rf_classifier.predict(reduced_test)
    time_exe += (time.time() - start_time)
    # calculate confusion matrix
    cm = confusion_matrix(y_test, y_pred)

    TP = cm.diagonal()
    FP = cm.sum(axis=0) - TP
    FN = cm.sum(axis=1) - TP
    TN = cm.sum() - (TP + FP + FN)

    total_TP += TP
    total_FP += FP
    total_FN += FN
    total_TN += TN

print("Thời gian chạy: --- %s giây ---" % time_exe )

average_TP = total_TP / n_folds
average_FP = total_FP / n_folds
average_FN = total_FN / n_folds
average_TN = total_TN / n_folds

print("Average TP:", average_TP)
print("Average FP:", average_FP)
print("Average FN:", average_FN)
print("Average TN:", average_TN)

Thời gian chạy: --- 0.4072606563568115 giây ---

Average TP: [231.5 387. ]

Average FP: [0.5 2.4]

Average FN: [2.4 0.5]

Average TN: [387.  231.5]


In [9]:
TP = average_TP[0]
FP = average_FP[0]
FN = average_FN[0]
TN = average_TN[0]

precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1_score = 2 * (precision * recall) / (precision + recall)



cnf_matrix = [[TP,FP],[FN,TN]]
TPR = TP / (TP + FN)
FPR = FP / (FP + TN)
FNR = FN / (FN + TP)
ACC = (TP+TN)/(TP+FP+TN+FN)
other_metrics = pd.DataFrame({'TPR': '%.4f' % TPR, 
                              'FPR': '%.4f' % FPR, 
                              'FNR': '%.4f' % FNR,
                              'ACC': '%.4f' % ACC,
                             'Precision':'%.4f' % precision,
                             'Recall':'%.4f' % recall,
                             'F1_score':'%.4f' % f1_score,}, index=[0]).to_string(col_space=9, index=False)
# print(clf_report)
print(cnf_matrix)
print(other_metrics)

[[231.5, 0.5], [2.4, 387.0]]

      TPR       FPR       FNR       ACC Precision    Recall  F1_score

   0.9897    0.0013    0.0103    0.9953    0.9978    0.9897    0.9938


## DT + IG

In [10]:
start_time = time.time()
vectorizer = TfidfVectorizer(min_df=0.0, analyzer="char", sublinear_tf=True, ngram_range=(2, 2), max_features=8000)
tfidf_matrix = vectorizer.fit_transform(train_data)

# feature_names = vectorizer.get_feature_names()

# feature_names


# pca
pca = SelectKBest(mutual_info_classif, k=256)
reduced_data = pca.fit_transform(tfidf_matrix.toarray(), label_train)

# random forest
rf_classifier = DecisionTreeClassifier()
rf_classifier.fit(reduced_data, label_train)

print("Thời gian chạy: ", time.time()-start_time)

Thời gian chạy:  324.608359336853


In [11]:
total_TP = 0
total_FP = 0
total_FN = 0
total_TN = 0
import time

time_exe = 0
for X_train, y_train, X_test, y_test in train_data_k_fold:
    # tf-idf
    tfidf_matrix = vectorizer.fit_transform(X_train)
    
    # pca
    reduced_data = pca.fit_transform(tfidf_matrix.toarray(), y_train)
    
    # random forest
    rf_classifier.fit(reduced_data, y_train)
    
    start_time = time.time()
    tfidf_test = vectorizer.transform(X_test)
    reduced_test  = pca.transform(tfidf_test.toarray())
    y_pred = rf_classifier.predict(reduced_test)
    time_exe += (time.time() - start_time)
    # calculate confusion matrix
    cm = confusion_matrix(y_test, y_pred)

    TP = cm.diagonal()
    FP = cm.sum(axis=0) - TP
    FN = cm.sum(axis=1) - TP
    TN = cm.sum() - (TP + FP + FN)

    total_TP += TP
    total_FP += FP
    total_FN += FN
    total_TN += TN

print("Thời gian chạy: --- %s giây ---" % time_exe )

average_TP = total_TP / n_folds
average_FP = total_FP / n_folds
average_FN = total_FN / n_folds
average_TN = total_TN / n_folds

print("Average TP:", average_TP)
print("Average FP:", average_FP)
print("Average FN:", average_FN)
print("Average TN:", average_TN)

Thời gian chạy: --- 0.28851747512817383 giây ---

Average TP: [230.1 384.7]

Average FP: [2.8 3.8]

Average FN: [3.8 2.8]

Average TN: [384.7 230.1]


In [12]:
TP = average_TP[0]
FP = average_FP[0]
FN = average_FN[0]
TN = average_TN[0]

precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1_score = 2 * (precision * recall) / (precision + recall)



cnf_matrix = [[TP,FP],[FN,TN]]
TPR = TP / (TP + FN)
FPR = FP / (FP + TN)
FNR = FN / (FN + TP)
ACC = (TP+TN)/(TP+FP+TN+FN)
other_metrics = pd.DataFrame({'TPR': '%.4f' % TPR, 
                              'FPR': '%.4f' % FPR, 
                              'FNR': '%.4f' % FNR,
                              'ACC': '%.4f' % ACC,
                             'Precision':'%.4f' % precision,
                             'Recall':'%.4f' % recall,
                             'F1_score':'%.4f' % f1_score,}, index=[0]).to_string(col_space=9, index=False)
# print(clf_report)
print(cnf_matrix)
print(other_metrics)

[[230.1, 2.8], [3.8, 384.7]]

      TPR       FPR       FNR       ACC Precision    Recall  F1_score

   0.9838    0.0072    0.0162    0.9894    0.9880    0.9838    0.9859


## SVM + IG

In [13]:
start_time = time.time()
vectorizer = TfidfVectorizer(min_df=0.0, analyzer="char", sublinear_tf=True, ngram_range=(2, 2), max_features=8000)
tfidf_matrix = vectorizer.fit_transform(train_data)

# feature_names = vectorizer.get_feature_names()

# feature_names


# pca
pca = SelectKBest(mutual_info_classif, k=256)
reduced_data = pca.fit_transform(tfidf_matrix.toarray(), label_train)

# random forest
rf_classifier = SVC()
rf_classifier.fit(reduced_data, label_train)

print("Thời gian chạy: ", time.time()-start_time)

Thời gian chạy:  339.6434488296509


In [14]:
total_TP = 0
total_FP = 0
total_FN = 0
total_TN = 0
import time

time_exe = 0
for X_train, y_train, X_test, y_test in train_data_k_fold:
    # tf-idf
    tfidf_matrix = vectorizer.fit_transform(X_train)
    
    # pca
    reduced_data = pca.fit_transform(tfidf_matrix.toarray(), y_train)
    
    # random forest
    rf_classifier.fit(reduced_data, y_train)
    
    start_time = time.time()
    tfidf_test = vectorizer.transform(X_test)
    reduced_test  = pca.transform(tfidf_test.toarray())
    y_pred = rf_classifier.predict(reduced_test)
    time_exe += (time.time() - start_time)
    # calculate confusion matrix
    cm = confusion_matrix(y_test, y_pred)

    TP = cm.diagonal()
    FP = cm.sum(axis=0) - TP
    FN = cm.sum(axis=1) - TP
    TN = cm.sum() - (TP + FP + FN)

    total_TP += TP
    total_FP += FP
    total_FN += FN
    total_TN += TN

print("Thời gian chạy: --- %s giây ---" % time_exe )

average_TP = total_TP / n_folds
average_FP = total_FP / n_folds
average_FN = total_FN / n_folds
average_TN = total_TN / n_folds

print("Average TP:", average_TP)
print("Average FP:", average_FP)
print("Average FN:", average_FN)
print("Average TN:", average_TN)

Thời gian chạy: --- 2.287877321243286 giây ---

Average TP: [226.4 387.4]

Average FP: [0.1 7.5]

Average FN: [7.5 0.1]

Average TN: [387.4 226.4]


In [15]:
TP = average_TP[0]
FP = average_FP[0]
FN = average_FN[0]
TN = average_TN[0]

precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1_score = 2 * (precision * recall) / (precision + recall)



cnf_matrix = [[TP,FP],[FN,TN]]
TPR = TP / (TP + FN)
FPR = FP / (FP + TN)
FNR = FN / (FN + TP)
ACC = (TP+TN)/(TP+FP+TN+FN)
other_metrics = pd.DataFrame({'TPR': '%.4f' % TPR, 
                              'FPR': '%.4f' % FPR, 
                              'FNR': '%.4f' % FNR,
                              'ACC': '%.4f' % ACC,
                             'Precision':'%.4f' % precision,
                             'Recall':'%.4f' % recall,
                             'F1_score':'%.4f' % f1_score,}, index=[0]).to_string(col_space=9, index=False)
# print(clf_report)
print(cnf_matrix)
print(other_metrics)

[[226.4, 0.1], [7.5, 387.4]]

      TPR       FPR       FNR       ACC Precision    Recall  F1_score

   0.9679    0.0003    0.0321    0.9878    0.9996    0.9679    0.9835


## NB + IG

In [16]:
start_time = time.time()
vectorizer = TfidfVectorizer(min_df=0.0, analyzer="char", sublinear_tf=True, ngram_range=(2, 2), max_features=8000)
tfidf_matrix = vectorizer.fit_transform(train_data)

# feature_names = vectorizer.get_feature_names()

# feature_names


# pca
pca = SelectKBest(mutual_info_classif, k=256)
reduced_data = pca.fit_transform(tfidf_matrix.toarray(), label_train)

# random forest
rf_classifier = GaussianNB()
rf_classifier.fit(reduced_data, label_train)

print("Thời gian chạy: ", time.time()-start_time)

Thời gian chạy:  323.91422033309937


In [17]:
total_TP = 0
total_FP = 0
total_FN = 0
total_TN = 0
import time

time_exe = 0
for X_train, y_train, X_test, y_test in train_data_k_fold:
    # tf-idf
    tfidf_matrix = vectorizer.fit_transform(X_train)
    
    # pca
    reduced_data = pca.fit_transform(tfidf_matrix.toarray(), y_train)
    
    # random forest
    rf_classifier.fit(reduced_data, y_train)
    
    start_time = time.time()
    tfidf_test = vectorizer.transform(X_test)
    reduced_test  = pca.transform(tfidf_test.toarray())
    y_pred = rf_classifier.predict(reduced_test)
    time_exe += (time.time() - start_time)
    # calculate confusion matrix
    cm = confusion_matrix(y_test, y_pred)

    TP = cm.diagonal()
    FP = cm.sum(axis=0) - TP
    FN = cm.sum(axis=1) - TP
    TN = cm.sum() - (TP + FP + FN)

    total_TP += TP
    total_FP += FP
    total_FN += FN
    total_TN += TN

print("Thời gian chạy: --- %s giây ---" % time_exe )

average_TP = total_TP / n_folds
average_FP = total_FP / n_folds
average_FN = total_FN / n_folds
average_TN = total_TN / n_folds

print("Average TP:", average_TP)
print("Average FP:", average_FP)
print("Average FN:", average_FN)
print("Average TN:", average_TN)

Thời gian chạy: --- 0.2943911552429199 giây ---

Average TP: [225.4 386.2]

Average FP: [1.3 8.5]

Average FN: [8.5 1.3]

Average TN: [386.2 225.4]


In [18]:
TP = average_TP[0]
FP = average_FP[0]
FN = average_FN[0]
TN = average_TN[0]

precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1_score = 2 * (precision * recall) / (precision + recall)



cnf_matrix = [[TP,FP],[FN,TN]]
TPR = TP / (TP + FN)
FPR = FP / (FP + TN)
FNR = FN / (FN + TP)
ACC = (TP+TN)/(TP+FP+TN+FN)
other_metrics = pd.DataFrame({'TPR': '%.4f' % TPR, 
                              'FPR': '%.4f' % FPR, 
                              'FNR': '%.4f' % FNR,
                              'ACC': '%.4f' % ACC,
                             'Precision':'%.4f' % precision,
                             'Recall':'%.4f' % recall,
                             'F1_score':'%.4f' % f1_score,}, index=[0]).to_string(col_space=9, index=False)
# print(clf_report)
print(cnf_matrix)
print(other_metrics)

[[225.4, 1.3], [8.5, 386.2]]

      TPR       FPR       FNR       ACC Precision    Recall  F1_score

   0.9637    0.0034    0.0363    0.9842    0.9943    0.9637    0.9787


# 3-gram

## RF + Information Gain

In [7]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif

In [10]:
start_time = time.time()
res_time = 0
vectorizer = TfidfVectorizer(min_df=0.0, analyzer="char", sublinear_tf=True, ngram_range=(3, 3), max_features=8000)
tfidf_matrix = vectorizer.fit_transform(train_data)
res_time += time.time()-start_time
# pca
pca = SelectKBest(mutual_info_classif, k=256)
reduced_data = pca.fit_transform(tfidf_matrix.toarray(), label_train)

# random forest
start_time = time.time()
rf_classifier = RandomForestClassifier(n_estimators=50)
rf_classifier.fit(reduced_data, label_train)
res_time += time.time()-start_time
print("Thời gian chạy: ", res_time)

Thời gian chạy:  3.5043678283691406


In [21]:
total_TP = 0
total_FP = 0
total_FN = 0
total_TN = 0
import time

time_exe = 0
for X_train, y_train, X_test, y_test in train_data_k_fold:
    # tf-idf
    tfidf_matrix = vectorizer.fit_transform(X_train)
    
    # pca
    reduced_data = pca.fit_transform(tfidf_matrix.toarray(), y_train)
    
    # random forest
    rf_classifier.fit(reduced_data, y_train)
    
    start_time = time.time()
    tfidf_test = vectorizer.transform(X_test)
    reduced_test  = pca.transform(tfidf_test.toarray())
    y_pred = rf_classifier.predict(reduced_test)
    time_exe += (time.time() - start_time)
    # calculate confusion matrix
    cm = confusion_matrix(y_test, y_pred)

    TP = cm.diagonal()
    FP = cm.sum(axis=0) - TP
    FN = cm.sum(axis=1) - TP
    TN = cm.sum() - (TP + FP + FN)

    total_TP += TP
    total_FP += FP
    total_FN += FN
    total_TN += TN

print("Thời gian chạy: --- %s giây ---" % time_exe )

average_TP = total_TP / n_folds
average_FP = total_FP / n_folds
average_FN = total_FN / n_folds
average_TN = total_TN / n_folds

print("Average TP:", average_TP)
print("Average FP:", average_FP)
print("Average FN:", average_FN)
print("Average TN:", average_TN)

Thời gian chạy: --- 0.6730127334594727 giây ---

Average TP: [222.  385.6]

Average FP: [ 1.9 11.9]

Average FN: [11.9  1.9]

Average TN: [385.6 222. ]


In [22]:
TP = average_TP[0]
FP = average_FP[0]
FN = average_FN[0]
TN = average_TN[0]

precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1_score = 2 * (precision * recall) / (precision + recall)



cnf_matrix = [[TP,FP],[FN,TN]]
TPR = TP / (TP + FN)
FPR = FP / (FP + TN)
FNR = FN / (FN + TP)
ACC = (TP+TN)/(TP+FP+TN+FN)
other_metrics = pd.DataFrame({'TPR': '%.4f' % TPR, 
                              'FPR': '%.4f' % FPR, 
                              'FNR': '%.4f' % FNR,
                              'ACC': '%.4f' % ACC,
                             'Precision':'%.4f' % precision,
                             'Recall':'%.4f' % recall,
                             'F1_score':'%.4f' % f1_score,}, index=[0]).to_string(col_space=9, index=False)
# print(clf_report)
print(cnf_matrix)
print(other_metrics)

[[222.0, 1.9], [11.9, 385.6]]

      TPR       FPR       FNR       ACC Precision    Recall  F1_score

   0.9491    0.0049    0.0509    0.9778    0.9915    0.9491    0.9699


## DT + InformationGain

In [23]:
start_time = time.time()
vectorizer = TfidfVectorizer(min_df=0.0, analyzer="char", sublinear_tf=True, ngram_range=(3, 3), max_features=8000)
tfidf_matrix = vectorizer.fit_transform(train_data)

# feature_names = vectorizer.get_feature_names()

# feature_names


# pca
pca = SelectKBest(mutual_info_classif, k=256)
reduced_data = pca.fit_transform(tfidf_matrix.toarray(), label_train)

# random forest
rf_classifier = DecisionTreeClassifier()
rf_classifier.fit(reduced_data, label_train)

print("Thời gian chạy: ", time.time()-start_time)

Thời gian chạy:  975.3322870731354


In [24]:
total_TP = 0
total_FP = 0
total_FN = 0
total_TN = 0
import time

time_exe = 0
for X_train, y_train, X_test, y_test in train_data_k_fold:
    # tf-idf
    tfidf_matrix = vectorizer.fit_transform(X_train)
    
    # pca
    reduced_data = pca.fit_transform(tfidf_matrix.toarray(), y_train)
    
    # random forest
    rf_classifier.fit(reduced_data, y_train)
    
    start_time = time.time()
    tfidf_test = vectorizer.transform(X_test)
    reduced_test  = pca.transform(tfidf_test.toarray())
    y_pred = rf_classifier.predict(reduced_test)
    time_exe += (time.time() - start_time)
    # calculate confusion matrix
    cm = confusion_matrix(y_test, y_pred)

    TP = cm.diagonal()
    FP = cm.sum(axis=0) - TP
    FN = cm.sum(axis=1) - TP
    TN = cm.sum() - (TP + FP + FN)

    total_TP += TP
    total_FP += FP
    total_FN += FN
    total_TN += TN

print("Thời gian chạy: --- %s giây ---" % time_exe )

average_TP = total_TP / n_folds
average_FP = total_FP / n_folds
average_FN = total_FN / n_folds
average_TN = total_TN / n_folds

print("Average TP:", average_TP)
print("Average FP:", average_FP)
print("Average FN:", average_FN)
print("Average TN:", average_TN)

Thời gian chạy: --- 0.5226876735687256 giây ---

Average TP: [220.8 384.9]

Average FP: [ 2.6 13.1]

Average FN: [13.1  2.6]

Average TN: [384.9 220.8]


In [25]:
TP = average_TP[0]
FP = average_FP[0]
FN = average_FN[0]
TN = average_TN[0]

precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1_score = 2 * (precision * recall) / (precision + recall)



cnf_matrix = [[TP,FP],[FN,TN]]
TPR = TP / (TP + FN)
FPR = FP / (FP + TN)
FNR = FN / (FN + TP)
ACC = (TP+TN)/(TP+FP+TN+FN)
other_metrics = pd.DataFrame({'TPR': '%.4f' % TPR, 
                              'FPR': '%.4f' % FPR, 
                              'FNR': '%.4f' % FNR,
                              'ACC': '%.4f' % ACC,
                             'Precision':'%.4f' % precision,
                             'Recall':'%.4f' % recall,
                             'F1_score':'%.4f' % f1_score,}, index=[0]).to_string(col_space=9, index=False)
# print(clf_report)
print(cnf_matrix)
print(other_metrics)

[[220.8, 2.6], [13.1, 384.9]]

      TPR       FPR       FNR       ACC Precision    Recall  F1_score

   0.9440    0.0067    0.0560    0.9747    0.9884    0.9440    0.9657


## SVM + InformationGain

In [26]:
start_time = time.time()
vectorizer = TfidfVectorizer(min_df=0.0, analyzer="char", sublinear_tf=True, ngram_range=(3, 3), max_features=8000)
tfidf_matrix = vectorizer.fit_transform(train_data)

# feature_names = vectorizer.get_feature_names()

# feature_names


# pca
pca = SelectKBest(mutual_info_classif, k=256)
reduced_data = pca.fit_transform(tfidf_matrix.toarray(), label_train)

# random forest
rf_classifier = SVC()
rf_classifier.fit(reduced_data, label_train)

print("Thời gian chạy: ", time.time()-start_time)

Thời gian chạy:  987.0398116111755


In [27]:
total_TP = 0
total_FP = 0
total_FN = 0
total_TN = 0
import time

time_exe = 0
for X_train, y_train, X_test, y_test in train_data_k_fold:
    # tf-idf
    tfidf_matrix = vectorizer.fit_transform(X_train)
    
    # pca
    reduced_data = pca.fit_transform(tfidf_matrix.toarray(), y_train)
    
    # random forest
    rf_classifier.fit(reduced_data, y_train)
    
    start_time = time.time()
    tfidf_test = vectorizer.transform(X_test)
    reduced_test  = pca.transform(tfidf_test.toarray())
    y_pred = rf_classifier.predict(reduced_test)
    time_exe += (time.time() - start_time)
    # calculate confusion matrix
    cm = confusion_matrix(y_test, y_pred)

    TP = cm.diagonal()
    FP = cm.sum(axis=0) - TP
    FN = cm.sum(axis=1) - TP
    TN = cm.sum() - (TP + FP + FN)

    total_TP += TP
    total_FP += FP
    total_FN += FN
    total_TN += TN

print("Thời gian chạy: --- %s giây ---" % time_exe )

average_TP = total_TP / n_folds
average_FP = total_FP / n_folds
average_FN = total_FN / n_folds
average_TN = total_TN / n_folds

print("Average TP:", average_TP)
print("Average FP:", average_FP)
print("Average FN:", average_FN)
print("Average TN:", average_TN)

Thời gian chạy: --- 2.065316677093506 giây ---

Average TP: [219.1 386.4]

Average FP: [ 1.1 14.8]

Average FN: [14.8  1.1]

Average TN: [386.4 219.1]


In [28]:
TP = average_TP[0]
FP = average_FP[0]
FN = average_FN[0]
TN = average_TN[0]

precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1_score = 2 * (precision * recall) / (precision + recall)



cnf_matrix = [[TP,FP],[FN,TN]]
TPR = TP / (TP + FN)
FPR = FP / (FP + TN)
FNR = FN / (FN + TP)
ACC = (TP+TN)/(TP+FP+TN+FN)
other_metrics = pd.DataFrame({'TPR': '%.4f' % TPR, 
                              'FPR': '%.4f' % FPR, 
                              'FNR': '%.4f' % FNR,
                              'ACC': '%.4f' % ACC,
                             'Precision':'%.4f' % precision,
                             'Recall':'%.4f' % recall,
                             'F1_score':'%.4f' % f1_score,}, index=[0]).to_string(col_space=9, index=False)
# print(clf_report)
print(cnf_matrix)
print(other_metrics)

[[219.1, 1.1], [14.8, 386.4]]

      TPR       FPR       FNR       ACC Precision    Recall  F1_score

   0.9367    0.0028    0.0633    0.9744    0.9950    0.9367    0.9650


## NB + InformationGain

In [29]:
start_time = time.time()
vectorizer = TfidfVectorizer(min_df=0.0, analyzer="char", sublinear_tf=True, ngram_range=(3, 3), max_features=8000)
tfidf_matrix = vectorizer.fit_transform(train_data)

# feature_names = vectorizer.get_feature_names()

# feature_names


# pca
pca = SelectKBest(mutual_info_classif, k=256)
reduced_data = pca.fit_transform(tfidf_matrix.toarray(), label_train)

# random forest
rf_classifier = GaussianNB()
rf_classifier.fit(reduced_data, label_train)

print("Thời gian chạy: ", time.time()-start_time)

Thời gian chạy:  973.6079137325287


In [30]:
total_TP = 0
total_FP = 0
total_FN = 0
total_TN = 0
import time

time_exe = 0
for X_train, y_train, X_test, y_test in train_data_k_fold:
    # tf-idf
    tfidf_matrix = vectorizer.fit_transform(X_train)
    
    # pca
    reduced_data = pca.fit_transform(tfidf_matrix.toarray(), y_train)
    
    # random forest
    rf_classifier.fit(reduced_data, y_train)
    
    start_time = time.time()
    tfidf_test = vectorizer.transform(X_test)
    reduced_test  = pca.transform(tfidf_test.toarray())
    y_pred = rf_classifier.predict(reduced_test)
    time_exe += (time.time() - start_time)
    # calculate confusion matrix
    cm = confusion_matrix(y_test, y_pred)

    TP = cm.diagonal()
    FP = cm.sum(axis=0) - TP
    FN = cm.sum(axis=1) - TP
    TN = cm.sum() - (TP + FP + FN)

    total_TP += TP
    total_FP += FP
    total_FN += FN
    total_TN += TN

print("Thời gian chạy: --- %s giây ---" % time_exe )

average_TP = total_TP / n_folds
average_FP = total_FP / n_folds
average_FN = total_FN / n_folds
average_TN = total_TN / n_folds

print("Average TP:", average_TP)
print("Average FP:", average_FP)
print("Average FN:", average_FN)
print("Average TN:", average_TN)

Thời gian chạy: --- 0.6419980525970459 giây ---

Average TP: [219.8 383.8]

Average FP: [ 3.7 14.1]

Average FN: [14.1  3.7]

Average TN: [383.8 219.8]


In [31]:
TP = average_TP[0]
FP = average_FP[0]
FN = average_FN[0]
TN = average_TN[0]

precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1_score = 2 * (precision * recall) / (precision + recall)



cnf_matrix = [[TP,FP],[FN,TN]]
TPR = TP / (TP + FN)
FPR = FP / (FP + TN)
FNR = FN / (FN + TP)
ACC = (TP+TN)/(TP+FP+TN+FN)
other_metrics = pd.DataFrame({'TPR': '%.4f' % TPR, 
                              'FPR': '%.4f' % FPR, 
                              'FNR': '%.4f' % FNR,
                              'ACC': '%.4f' % ACC,
                             'Precision':'%.4f' % precision,
                             'Recall':'%.4f' % recall,
                             'F1_score':'%.4f' % f1_score,}, index=[0]).to_string(col_space=9, index=False)
# print(clf_report)
print(cnf_matrix)
print(other_metrics)

[[219.8, 3.7], [14.1, 383.8]]

      TPR       FPR       FNR       ACC Precision    Recall  F1_score

   0.9397    0.0095    0.0603    0.9714    0.9834    0.9397    0.9611


# 4-gram

## RF + IG

In [32]:
start_time = time.time()
vectorizer = TfidfVectorizer(min_df=0.0, analyzer="char", sublinear_tf=True, ngram_range=(4, 4), max_features=8000)
tfidf_matrix = vectorizer.fit_transform(train_data)
# pca
pca = SelectKBest(mutual_info_classif, k=256)
reduced_data = pca.fit_transform(tfidf_matrix.toarray(), label_train)

# random forest
rf_classifier = RandomForestClassifier(n_estimators=50)
rf_classifier.fit(reduced_data, label_train)
print("Thời gian chạy: ", time.time()-start_time)

Thời gian chạy:  976.0084745883942


In [33]:
total_TP = 0
total_FP = 0
total_FN = 0
total_TN = 0
import time

time_exe = 0
for X_train, y_train, X_test, y_test in train_data_k_fold:
    # tf-idf
    tfidf_matrix = vectorizer.fit_transform(X_train)
    
    # pca
    reduced_data = pca.fit_transform(tfidf_matrix.toarray(), y_train)
    
    # random forest
    rf_classifier.fit(reduced_data, y_train)
    
    start_time = time.time()
    tfidf_test = vectorizer.transform(X_test)
    reduced_test  = pca.transform(tfidf_test.toarray())
    y_pred = rf_classifier.predict(reduced_test)
    time_exe += (time.time() - start_time)
    # calculate confusion matrix
    cm = confusion_matrix(y_test, y_pred)

    TP = cm.diagonal()
    FP = cm.sum(axis=0) - TP
    FN = cm.sum(axis=1) - TP
    TN = cm.sum() - (TP + FP + FN)

    total_TP += TP
    total_FP += FP
    total_FN += FN
    total_TN += TN

print("Thời gian chạy: --- %s giây ---" % time_exe )

average_TP = total_TP / n_folds
average_FP = total_FP / n_folds
average_FN = total_FN / n_folds
average_TN = total_TN / n_folds

print("Average TP:", average_TP)
print("Average FP:", average_FP)
print("Average FN:", average_FN)
print("Average TN:", average_TN)

Thời gian chạy: --- 0.6694643497467041 giây ---

Average TP: [220.1 387.2]

Average FP: [ 0.3 13.8]

Average FN: [13.8  0.3]

Average TN: [387.2 220.1]


In [34]:
TP = average_TP[0]
FP = average_FP[0]
FN = average_FN[0]
TN = average_TN[0]

precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1_score = 2 * (precision * recall) / (precision + recall)



cnf_matrix = [[TP,FP],[FN,TN]]
TPR = TP / (TP + FN)
FPR = FP / (FP + TN)
FNR = FN / (FN + TP)
ACC = (TP+TN)/(TP+FP+TN+FN)
other_metrics = pd.DataFrame({'TPR': '%.4f' % TPR, 
                              'FPR': '%.4f' % FPR, 
                              'FNR': '%.4f' % FNR,
                              'ACC': '%.4f' % ACC,
                             'Precision':'%.4f' % precision,
                             'Recall':'%.4f' % recall,
                             'F1_score':'%.4f' % f1_score,}, index=[0]).to_string(col_space=9, index=False)
# print(clf_report)
print(cnf_matrix)
print(other_metrics)

[[220.1, 0.3], [13.8, 387.2]]

      TPR       FPR       FNR       ACC Precision    Recall  F1_score

   0.9410    0.0008    0.0590    0.9773    0.9986    0.9410    0.9690


## DT + IG

In [35]:
start_time = time.time()
vectorizer = TfidfVectorizer(min_df=0.0, analyzer="char", sublinear_tf=True, ngram_range=(4, 4), max_features=8000)
tfidf_matrix = vectorizer.fit_transform(train_data)

# feature_names = vectorizer.get_feature_names()

# feature_names


# pca
pca = SelectKBest(mutual_info_classif, k=256)
reduced_data = pca.fit_transform(tfidf_matrix.toarray(), label_train)

# random forest
rf_classifier = DecisionTreeClassifier()
rf_classifier.fit(reduced_data, label_train)

print("Thời gian chạy: ", time.time()-start_time)

Thời gian chạy:  974.9163873195648


In [36]:
total_TP = 0
total_FP = 0
total_FN = 0
total_TN = 0
import time

time_exe = 0
for X_train, y_train, X_test, y_test in train_data_k_fold:
    # tf-idf
    tfidf_matrix = vectorizer.fit_transform(X_train)
    
    # pca
    reduced_data = pca.fit_transform(tfidf_matrix.toarray(), y_train)
    
    # random forest
    rf_classifier.fit(reduced_data, y_train)
    
    start_time = time.time()
    tfidf_test = vectorizer.transform(X_test)
    reduced_test  = pca.transform(tfidf_test.toarray())
    y_pred = rf_classifier.predict(reduced_test)
    time_exe += (time.time() - start_time)
    # calculate confusion matrix
    cm = confusion_matrix(y_test, y_pred)

    TP = cm.diagonal()
    FP = cm.sum(axis=0) - TP
    FN = cm.sum(axis=1) - TP
    TN = cm.sum() - (TP + FP + FN)

    total_TP += TP
    total_FP += FP
    total_FN += FN
    total_TN += TN

print("Thời gian chạy: --- %s giây ---" % time_exe )

average_TP = total_TP / n_folds
average_FP = total_FP / n_folds
average_FN = total_FN / n_folds
average_TN = total_TN / n_folds

print("Average TP:", average_TP)
print("Average FP:", average_FP)
print("Average FN:", average_FN)
print("Average TN:", average_TN)

Thời gian chạy: --- 0.48395586013793945 giây ---

Average TP: [220.6 387.2]

Average FP: [ 0.3 13.3]

Average FN: [13.3  0.3]

Average TN: [387.2 220.6]


In [37]:
TP = average_TP[0]
FP = average_FP[0]
FN = average_FN[0]
TN = average_TN[0]

precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1_score = 2 * (precision * recall) / (precision + recall)



cnf_matrix = [[TP,FP],[FN,TN]]
TPR = TP / (TP + FN)
FPR = FP / (FP + TN)
FNR = FN / (FN + TP)
ACC = (TP+TN)/(TP+FP+TN+FN)
other_metrics = pd.DataFrame({'TPR': '%.4f' % TPR, 
                              'FPR': '%.4f' % FPR, 
                              'FNR': '%.4f' % FNR,
                              'ACC': '%.4f' % ACC,
                             'Precision':'%.4f' % precision,
                             'Recall':'%.4f' % recall,
                             'F1_score':'%.4f' % f1_score,}, index=[0]).to_string(col_space=9, index=False)
# print(clf_report)
print(cnf_matrix)
print(other_metrics)

[[220.6, 0.3], [13.3, 387.2]]

      TPR       FPR       FNR       ACC Precision    Recall  F1_score

   0.9431    0.0008    0.0569    0.9781    0.9986    0.9431    0.9701


## SVM + IG

In [38]:
start_time = time.time()
vectorizer = TfidfVectorizer(min_df=0.0, analyzer="char", sublinear_tf=True, ngram_range=(4, 4), max_features=8000)
tfidf_matrix = vectorizer.fit_transform(train_data)

# feature_names = vectorizer.get_feature_names()

# feature_names


# pca
pca = SelectKBest(mutual_info_classif, k=256)
reduced_data = pca.fit_transform(tfidf_matrix.toarray(), label_train)

# random forest
rf_classifier = SVC()
rf_classifier.fit(reduced_data, label_train)

print("Thời gian chạy: ", time.time()-start_time)

Thời gian chạy:  997.3429307937622


In [39]:
total_TP = 0
total_FP = 0
total_FN = 0
total_TN = 0
import time

time_exe = 0
for X_train, y_train, X_test, y_test in train_data_k_fold:
    # tf-idf
    tfidf_matrix = vectorizer.fit_transform(X_train)
    
    # pca
    reduced_data = pca.fit_transform(tfidf_matrix.toarray(), y_train)
    
    # random forest
    rf_classifier.fit(reduced_data, y_train)
    
    start_time = time.time()
    tfidf_test = vectorizer.transform(X_test)
    reduced_test  = pca.transform(tfidf_test.toarray())
    y_pred = rf_classifier.predict(reduced_test)
    time_exe += (time.time() - start_time)
    # calculate confusion matrix
    cm = confusion_matrix(y_test, y_pred)

    TP = cm.diagonal()
    FP = cm.sum(axis=0) - TP
    FN = cm.sum(axis=1) - TP
    TN = cm.sum() - (TP + FP + FN)

    total_TP += TP
    total_FP += FP
    total_FN += FN
    total_TN += TN

print("Thời gian chạy: --- %s giây ---" % time_exe )

average_TP = total_TP / n_folds
average_FP = total_FP / n_folds
average_FN = total_FN / n_folds
average_TN = total_TN / n_folds

print("Average TP:", average_TP)
print("Average FP:", average_FP)
print("Average FN:", average_FN)
print("Average TN:", average_TN)

Thời gian chạy: --- 1.5194182395935059 giây ---

Average TP: [218.4 385.4]

Average FP: [ 2.1 15.5]

Average FN: [15.5  2.1]

Average TN: [385.4 218.4]


In [40]:
TP = average_TP[0]
FP = average_FP[0]
FN = average_FN[0]
TN = average_TN[0]

precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1_score = 2 * (precision * recall) / (precision + recall)



cnf_matrix = [[TP,FP],[FN,TN]]
TPR = TP / (TP + FN)
FPR = FP / (FP + TN)
FNR = FN / (FN + TP)
ACC = (TP+TN)/(TP+FP+TN+FN)
other_metrics = pd.DataFrame({'TPR': '%.4f' % TPR, 
                              'FPR': '%.4f' % FPR, 
                              'FNR': '%.4f' % FNR,
                              'ACC': '%.4f' % ACC,
                             'Precision':'%.4f' % precision,
                             'Recall':'%.4f' % recall,
                             'F1_score':'%.4f' % f1_score,}, index=[0]).to_string(col_space=9, index=False)
# print(clf_report)
print(cnf_matrix)
print(other_metrics)

[[218.4, 2.1], [15.5, 385.4]]

      TPR       FPR       FNR       ACC Precision    Recall  F1_score

   0.9337    0.0054    0.0663    0.9717    0.9905    0.9337    0.9613


## NB + IG

In [41]:
start_time = time.time()
vectorizer = TfidfVectorizer(min_df=0.0, analyzer="char", sublinear_tf=True, ngram_range=(4, 4), max_features=8000)
tfidf_matrix = vectorizer.fit_transform(train_data)

# feature_names = vectorizer.get_feature_names()

# feature_names


# pca
pca = SelectKBest(mutual_info_classif, k=256)
reduced_data = pca.fit_transform(tfidf_matrix.toarray(), label_train)

# random forest
rf_classifier = GaussianNB()
rf_classifier.fit(reduced_data, label_train)

print("Thời gian chạy: ", time.time()-start_time)

Thời gian chạy:  985.1868133544922


In [42]:
total_TP = 0
total_FP = 0
total_FN = 0
total_TN = 0
import time

time_exe = 0
for X_train, y_train, X_test, y_test in train_data_k_fold:
    # tf-idf
    tfidf_matrix = vectorizer.fit_transform(X_train)
    
    # pca
    reduced_data = pca.fit_transform(tfidf_matrix.toarray(), y_train)
    
    # random forest
    rf_classifier.fit(reduced_data, y_train)
    
    start_time = time.time()
    tfidf_test = vectorizer.transform(X_test)
    reduced_test  = pca.transform(tfidf_test.toarray())
    y_pred = rf_classifier.predict(reduced_test)
    time_exe += (time.time() - start_time)
    # calculate confusion matrix
    cm = confusion_matrix(y_test, y_pred)

    TP = cm.diagonal()
    FP = cm.sum(axis=0) - TP
    FN = cm.sum(axis=1) - TP
    TN = cm.sum() - (TP + FP + FN)

    total_TP += TP
    total_FP += FP
    total_FN += FN
    total_TN += TN

print("Thời gian chạy: --- %s giây ---" % time_exe )

average_TP = total_TP / n_folds
average_FP = total_FP / n_folds
average_FN = total_FN / n_folds
average_TN = total_TN / n_folds

print("Average TP:", average_TP)
print("Average FP:", average_FP)
print("Average FN:", average_FN)
print("Average TN:", average_TN)

Thời gian chạy: --- 0.5677294731140137 giây ---

Average TP: [216.8 385.3]

Average FP: [ 2.2 17.1]

Average FN: [17.1  2.2]

Average TN: [385.3 216.8]


In [43]:
TP = average_TP[0]
FP = average_FP[0]
FN = average_FN[0]
TN = average_TN[0]

precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1_score = 2 * (precision * recall) / (precision + recall)



cnf_matrix = [[TP,FP],[FN,TN]]
TPR = TP / (TP + FN)
FPR = FP / (FP + TN)
FNR = FN / (FN + TP)
ACC = (TP+TN)/(TP+FP+TN+FN)
other_metrics = pd.DataFrame({'TPR': '%.4f' % TPR, 
                              'FPR': '%.4f' % FPR, 
                              'FNR': '%.4f' % FNR,
                              'ACC': '%.4f' % ACC,
                             'Precision':'%.4f' % precision,
                             'Recall':'%.4f' % recall,
                             'F1_score':'%.4f' % f1_score,}, index=[0]).to_string(col_space=9, index=False)
# print(clf_report)
print(cnf_matrix)
print(other_metrics)

[[216.8, 2.2], [17.1, 385.3]]

      TPR       FPR       FNR       ACC Precision    Recall  F1_score

   0.9269    0.0057    0.0731    0.9689    0.9900    0.9269    0.9574


# 5-gram

## RF + IG

In [44]:
start_time = time.time()
vectorizer = TfidfVectorizer(min_df=0.0, analyzer="char", sublinear_tf=True, ngram_range=(5, 5), max_features=8000)
tfidf_matrix = vectorizer.fit_transform(train_data)
# pca
pca = SelectKBest(mutual_info_classif, k=256)
reduced_data = pca.fit_transform(tfidf_matrix.toarray(), label_train)

# random forest
rf_classifier = RandomForestClassifier(n_estimators=50)
rf_classifier.fit(reduced_data, label_train)
print("Thời gian chạy: ", time.time()-start_time)

Thời gian chạy:  972.8106889724731


In [45]:
total_TP = 0
total_FP = 0
total_FN = 0
total_TN = 0
import time

time_exe = 0
for X_train, y_train, X_test, y_test in train_data_k_fold:
    # tf-idf
    tfidf_matrix = vectorizer.fit_transform(X_train)
    
    # pca
    reduced_data = pca.fit_transform(tfidf_matrix.toarray(), y_train)
    
    # random forest
    rf_classifier.fit(reduced_data, y_train)
    
    start_time = time.time()
    tfidf_test = vectorizer.transform(X_test)
    reduced_test  = pca.transform(tfidf_test.toarray())
    y_pred = rf_classifier.predict(reduced_test)
    time_exe += (time.time() - start_time)
    # calculate confusion matrix
    cm = confusion_matrix(y_test, y_pred)

    TP = cm.diagonal()
    FP = cm.sum(axis=0) - TP
    FN = cm.sum(axis=1) - TP
    TN = cm.sum() - (TP + FP + FN)

    total_TP += TP
    total_FP += FP
    total_FN += FN
    total_TN += TN

print("Thời gian chạy: --- %s giây ---" % time_exe )

average_TP = total_TP / n_folds
average_FP = total_FP / n_folds
average_FN = total_FN / n_folds
average_TN = total_TN / n_folds

print("Average TP:", average_TP)
print("Average FP:", average_FP)
print("Average FN:", average_FN)
print("Average TN:", average_TN)

Thời gian chạy: --- 0.5185937881469727 giây ---

Average TP: [213.9 387.3]

Average FP: [ 0.2 20. ]

Average FN: [20.   0.2]

Average TN: [387.3 213.9]


In [46]:
TP = average_TP[0]
FP = average_FP[0]
FN = average_FN[0]
TN = average_TN[0]

precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1_score = 2 * (precision * recall) / (precision + recall)



cnf_matrix = [[TP,FP],[FN,TN]]
TPR = TP / (TP + FN)
FPR = FP / (FP + TN)
FNR = FN / (FN + TP)
ACC = (TP+TN)/(TP+FP+TN+FN)
other_metrics = pd.DataFrame({'TPR': '%.4f' % TPR, 
                              'FPR': '%.4f' % FPR, 
                              'FNR': '%.4f' % FNR,
                              'ACC': '%.4f' % ACC,
                             'Precision':'%.4f' % precision,
                             'Recall':'%.4f' % recall,
                             'F1_score':'%.4f' % f1_score,}, index=[0]).to_string(col_space=9, index=False)
# print(clf_report)
print(cnf_matrix)
print(other_metrics)

[[213.9, 0.2], [20.0, 387.3]]

      TPR       FPR       FNR       ACC Precision    Recall  F1_score

   0.9145    0.0005    0.0855    0.9675    0.9991    0.9145    0.9549


## DT + IG

In [47]:
start_time = time.time()
vectorizer = TfidfVectorizer(min_df=0.0, analyzer="char", sublinear_tf=True, ngram_range=(5, 5), max_features=8000)
tfidf_matrix = vectorizer.fit_transform(train_data)

# feature_names = vectorizer.get_feature_names()

# feature_names


# pca
pca = SelectKBest(mutual_info_classif, k=256)
reduced_data = pca.fit_transform(tfidf_matrix.toarray(), label_train)

# random forest
rf_classifier = DecisionTreeClassifier()
rf_classifier.fit(reduced_data, label_train)

print("Thời gian chạy: ", time.time()-start_time)

Thời gian chạy:  974.2672214508057


In [48]:
total_TP = 0
total_FP = 0
total_FN = 0
total_TN = 0
import time

time_exe = 0
for X_train, y_train, X_test, y_test in train_data_k_fold:
    # tf-idf
    tfidf_matrix = vectorizer.fit_transform(X_train)
    
    # pca
    reduced_data = pca.fit_transform(tfidf_matrix.toarray(), y_train)
    
    # random forest
    rf_classifier.fit(reduced_data, y_train)
    
    start_time = time.time()
    tfidf_test = vectorizer.transform(X_test)
    reduced_test  = pca.transform(tfidf_test.toarray())
    y_pred = rf_classifier.predict(reduced_test)
    time_exe += (time.time() - start_time)
    # calculate confusion matrix
    cm = confusion_matrix(y_test, y_pred)

    TP = cm.diagonal()
    FP = cm.sum(axis=0) - TP
    FN = cm.sum(axis=1) - TP
    TN = cm.sum() - (TP + FP + FN)

    total_TP += TP
    total_FP += FP
    total_FN += FN
    total_TN += TN

print("Thời gian chạy: --- %s giây ---" % time_exe )

average_TP = total_TP / n_folds
average_FP = total_FP / n_folds
average_FN = total_FN / n_folds
average_TN = total_TN / n_folds

print("Average TP:", average_TP)
print("Average FP:", average_FP)
print("Average FN:", average_FN)
print("Average TN:", average_TN)

Thời gian chạy: --- 0.4377598762512207 giây ---

Average TP: [214.2 387.2]

Average FP: [ 0.3 19.7]

Average FN: [19.7  0.3]

Average TN: [387.2 214.2]


In [49]:
TP = average_TP[0]
FP = average_FP[0]
FN = average_FN[0]
TN = average_TN[0]

precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1_score = 2 * (precision * recall) / (precision + recall)



cnf_matrix = [[TP,FP],[FN,TN]]
TPR = TP / (TP + FN)
FPR = FP / (FP + TN)
FNR = FN / (FN + TP)
ACC = (TP+TN)/(TP+FP+TN+FN)
other_metrics = pd.DataFrame({'TPR': '%.4f' % TPR, 
                              'FPR': '%.4f' % FPR, 
                              'FNR': '%.4f' % FNR,
                              'ACC': '%.4f' % ACC,
                             'Precision':'%.4f' % precision,
                             'Recall':'%.4f' % recall,
                             'F1_score':'%.4f' % f1_score,}, index=[0]).to_string(col_space=9, index=False)
# print(clf_report)
print(cnf_matrix)
print(other_metrics)

[[214.2, 0.3], [19.7, 387.2]]

      TPR       FPR       FNR       ACC Precision    Recall  F1_score

   0.9158    0.0008    0.0842    0.9678    0.9986    0.9158    0.9554


## SVM + IG

In [50]:
start_time = time.time()
vectorizer = TfidfVectorizer(min_df=0.0, analyzer="char", sublinear_tf=True, ngram_range=(5, 5), max_features=8000)
tfidf_matrix = vectorizer.fit_transform(train_data)

# feature_names = vectorizer.get_feature_names()

# feature_names


# pca
pca = SelectKBest(mutual_info_classif, k=256)
reduced_data = pca.fit_transform(tfidf_matrix.toarray(), label_train)

# random forest
rf_classifier = SVC()
rf_classifier.fit(reduced_data, label_train)

print("Thời gian chạy: ", time.time()-start_time)

Thời gian chạy:  989.1377437114716


In [51]:
total_TP = 0
total_FP = 0
total_FN = 0
total_TN = 0
import time

time_exe = 0
for X_train, y_train, X_test, y_test in train_data_k_fold:
    # tf-idf
    tfidf_matrix = vectorizer.fit_transform(X_train)
    
    # pca
    reduced_data = pca.fit_transform(tfidf_matrix.toarray(), y_train)
    
    # random forest
    rf_classifier.fit(reduced_data, y_train)
    
    start_time = time.time()
    tfidf_test = vectorizer.transform(X_test)
    reduced_test  = pca.transform(tfidf_test.toarray())
    y_pred = rf_classifier.predict(reduced_test)
    time_exe += (time.time() - start_time)
    # calculate confusion matrix
    cm = confusion_matrix(y_test, y_pred)

    TP = cm.diagonal()
    FP = cm.sum(axis=0) - TP
    FN = cm.sum(axis=1) - TP
    TN = cm.sum() - (TP + FP + FN)

    total_TP += TP
    total_FP += FP
    total_FN += FN
    total_TN += TN

print("Thời gian chạy: --- %s giây ---" % time_exe )

average_TP = total_TP / n_folds
average_FP = total_FP / n_folds
average_FN = total_FN / n_folds
average_TN = total_TN / n_folds

print("Average TP:", average_TP)
print("Average FP:", average_FP)
print("Average FN:", average_FN)
print("Average TN:", average_TN)

Thời gian chạy: --- 1.2238032817840576 giây ---

Average TP: [214.6 386.6]

Average FP: [ 0.9 19.3]

Average FN: [19.3  0.9]

Average TN: [386.6 214.6]


In [52]:
TP = average_TP[0]
FP = average_FP[0]
FN = average_FN[0]
TN = average_TN[0]

precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1_score = 2 * (precision * recall) / (precision + recall)



cnf_matrix = [[TP,FP],[FN,TN]]
TPR = TP / (TP + FN)
FPR = FP / (FP + TN)
FNR = FN / (FN + TP)
ACC = (TP+TN)/(TP+FP+TN+FN)
other_metrics = pd.DataFrame({'TPR': '%.4f' % TPR, 
                              'FPR': '%.4f' % FPR, 
                              'FNR': '%.4f' % FNR,
                              'ACC': '%.4f' % ACC,
                             'Precision':'%.4f' % precision,
                             'Recall':'%.4f' % recall,
                             'F1_score':'%.4f' % f1_score,}, index=[0]).to_string(col_space=9, index=False)
# print(clf_report)
print(cnf_matrix)
print(other_metrics)

[[214.6, 0.9], [19.3, 386.6]]

      TPR       FPR       FNR       ACC Precision    Recall  F1_score

   0.9175    0.0023    0.0825    0.9675    0.9958    0.9175    0.9551


## NB + IG

In [53]:
start_time = time.time()
vectorizer = TfidfVectorizer(min_df=0.0, analyzer="char", sublinear_tf=True, ngram_range=(5, 5), max_features=8000)
tfidf_matrix = vectorizer.fit_transform(train_data)

# feature_names = vectorizer.get_feature_names()

# feature_names


# pca
pca = SelectKBest(mutual_info_classif, k=256)
reduced_data = pca.fit_transform(tfidf_matrix.toarray(), label_train)

# random forest
rf_classifier = GaussianNB()
rf_classifier.fit(reduced_data, label_train)

print("Thời gian chạy: ", time.time()-start_time)

Thời gian chạy:  980.6711821556091


In [54]:
total_TP = 0
total_FP = 0
total_FN = 0
total_TN = 0
import time

time_exe = 0
for X_train, y_train, X_test, y_test in train_data_k_fold:
    # tf-idf
    tfidf_matrix = vectorizer.fit_transform(X_train)
    
    # pca
    reduced_data = pca.fit_transform(tfidf_matrix.toarray(), y_train)
    
    # random forest
    rf_classifier.fit(reduced_data, y_train)
    
    start_time = time.time()
    tfidf_test = vectorizer.transform(X_test)
    reduced_test  = pca.transform(tfidf_test.toarray())
    y_pred = rf_classifier.predict(reduced_test)
    time_exe += (time.time() - start_time)
    # calculate confusion matrix
    cm = confusion_matrix(y_test, y_pred)

    TP = cm.diagonal()
    FP = cm.sum(axis=0) - TP
    FN = cm.sum(axis=1) - TP
    TN = cm.sum() - (TP + FP + FN)

    total_TP += TP
    total_FP += FP
    total_FN += FN
    total_TN += TN

print("Thời gian chạy: --- %s giây ---" % time_exe )

average_TP = total_TP / n_folds
average_FP = total_FP / n_folds
average_FN = total_FN / n_folds
average_TN = total_TN / n_folds

print("Average TP:", average_TP)
print("Average FP:", average_FP)
print("Average FN:", average_FN)
print("Average TN:", average_TN)

Thời gian chạy: --- 0.5614137649536133 giây ---

Average TP: [214.7 387.1]

Average FP: [ 0.4 19.2]

Average FN: [19.2  0.4]

Average TN: [387.1 214.7]


In [55]:
TP = average_TP[0]
FP = average_FP[0]
FN = average_FN[0]
TN = average_TN[0]

precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1_score = 2 * (precision * recall) / (precision + recall)



cnf_matrix = [[TP,FP],[FN,TN]]
TPR = TP / (TP + FN)
FPR = FP / (FP + TN)
FNR = FN / (FN + TP)
ACC = (TP+TN)/(TP+FP+TN+FN)
other_metrics = pd.DataFrame({'TPR': '%.4f' % TPR, 
                              'FPR': '%.4f' % FPR, 
                              'FNR': '%.4f' % FNR,
                              'ACC': '%.4f' % ACC,
                             'Precision':'%.4f' % precision,
                             'Recall':'%.4f' % recall,
                             'F1_score':'%.4f' % f1_score,}, index=[0]).to_string(col_space=9, index=False)
# print(clf_report)
print(cnf_matrix)
print(other_metrics)

[[214.7, 0.4], [19.2, 387.1]]

      TPR       FPR       FNR       ACC Precision    Recall  F1_score

   0.9179    0.0010    0.0821    0.9685    0.9981    0.9179    0.9563
